In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt

In [2]:
start_date = '2016'
end_date = '2018'

### Weather ds

In [3]:
weatherdf = pd.read_csv('WeatherEvents_Jan2016-Dec2021.csv')

In [4]:
weatherdf.head(2)

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),Precipitation(in),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode
0,W-1,Snow,Light,2016-01-06 23:14:00,2016-01-07 00:34:00,0.0,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
1,W-2,Snow,Light,2016-01-07 04:14:00,2016-01-07 04:54:00,0.0,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0


In [5]:
mask = (weatherdf['StartTime(UTC)'] > start_date) & (weatherdf['StartTime(UTC)'] <= end_date)
weatherdf = weatherdf.loc[mask]

In [6]:
weatherdf['Month'] = pd.DatetimeIndex(weatherdf['StartTime(UTC)']).month_name()

In [7]:
weatherdf = weatherdf.groupby('Month', as_index = False)['Precipitation(in)'].sum()

In [8]:
weatherdf=weatherdf.rename(columns = {'Precipitation(in)':'Precipitation'})

In [32]:
weatherdf.head(12)

,Month,Precipitation
0,April,25595.16
1,August,28942.84
2,December,18506.39
3,February,14756.89
4,January,23398.14
5,July,26699.05
6,June,26542.57
7,March,21972.40
8,May,27319.81
9,November,14435.62


### Accidents ds

In [10]:
accidentsdf = pd.read_csv('./DATA/US_Accidents_June20_Cleaned.csv')

In [11]:
accidentsdf.head(2)

,Unnamed: 0,ID,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance_mi,Description,...,Traffic_Calming,Traffic_Signal,Sunrise_Sunset,Rain_Bool,Snow_Bool,Thunder_Bool,Wind_Bool,Fog_Bool,Hail_Bool,Sand_Dust_Bool
0,0,A-1,201.0,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,...,False,False,1.0,True,False,False,False,False,False,False
1,1,A-2,201.0,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,0.01,Accident on Brice Rd at Tussing Rd. Expect del...,...,False,False,1.0,True,False,False,False,False,False,False


In [12]:
mask = (accidentsdf['Start_Time'] > start_date) & (accidentsdf['Start_Time'] <= end_date)
accidentsdf = accidentsdf.loc[mask]

In [13]:
accidentsdf['Month'] = pd.DatetimeIndex(accidentsdf['Start_Time']).month_name()

In [14]:
accidentsdf = accidentsdf.groupby('Month', as_index = False)['ID'].size()

In [15]:
accidentsdf=accidentsdf.rename(columns = {'size':'No_accidents'})

In [31]:
accidentsdf.head(12)

,Month,No_accidents
0,April,66135
1,August,137155
2,December,131014
3,February,52603
4,January,55655
5,July,88922
6,June,76695
7,March,63579
8,May,58691
9,November,134963


#### Sum license

In [17]:
licensedf = pd.read_csv('./DATA/license_data.csv')

In [18]:
licensedf.head()

,State,Year,Gender,Age,Drivers
0,Alabama,2010,Female,15-,15050
1,Alabama,2010,Female,16,22814
2,Alabama,2010,Female,17,25723
3,Alabama,2010,Female,18,27903
4,Alabama,2010,Female,19-24,192651


In [19]:
licensedf = licensedf[(licensedf.Year.isin([2016, 2017, 2018]))]

In [21]:
tempdf = licensedf.groupby('Year', as_index = False)['Drivers'].sum()

In [22]:
tempdf.head()

,Year,Drivers
0,2016,221994424
1,2017,225346257
2,2018,226065619


In [23]:
totalDrivers = tempdf['Drivers'].sum()

In [24]:
totalDrivers

673406300

### Merge

In [25]:
acc_wea_dri = pd.merge(weatherdf, accidentsdf, on='Month')

In [30]:
acc_wea_dri.head(12)

,Month,Precipitation,No_accidents
0,April,25595.16,66135
1,August,28942.84,137155
2,December,18506.39,131014
3,February,14756.89,52603
4,January,23398.14,55655
5,July,26699.05,88922
6,June,26542.57,76695
7,March,21972.40,63579
8,May,27319.81,58691
9,November,14435.62,134963


In [35]:
acc_wea_dri['acc_per_driver'] = acc_wea_dri['No_accidents']/totalDrivers*100

In [36]:
acc_wea_dri.head()

,Month,Precipitation,No_accidents,acc_per_driver
0,April,25595.16,66135,0.009821
1,August,28942.84,137155,0.020367
2,December,18506.39,131014,0.019455
3,February,14756.89,52603,0.007811
4,January,23398.14,55655,0.008265


In [37]:
acc_wea_dri.to_csv('acc_wea_dri.csv')